# Grid-Search und Parameter-Tuning

Du hast nun bereits Gütekriterien für Machine Learning-Verfahren kennengelernt und mit der Kreuzvalidierung gearbeitet, um die Stabilität der Modelle bewerten zu können.

Die Verfahren haben allerdings noch mehr Parameter. So kannst du auch an der Vektorisierung Änderungen vornehmen oder überlegen, welches überhaupt geeignete Textfelder für die Klassifikation sind. Dies wird unter dem Begriff *Hyperparameter* zusammengefasst, die du jetzt optimieren wirst.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In diesem Teil betrachtest du nur die Klassifikation nach Autoren. Genauso kannst du das natürlich für Keywords (einzeln!) oder für die Kommentare durchführen:

## Daten für Autoren-Klassifikation vorbereiten

Diesen Teil kennst du schon:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

min_articles = min(top_authors["title"])
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

Den Split in Trainings- und Testdaten führt der Algorithmus später selbst durch.

## Pipeline und Parameter Range

Weil du Parameter sowohl in der Vektorisierung als auch im Modell optimieren möchtest, fasst du das in einer sog. `Pipeline` zusammen. Die Pipeline erhält als Input die Texte und gibt dir als Ergebnis den Autor.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.linear_model import SGDClassifier

text_pipe = Pipeline([("vect", TfidfVectorizer(stop_words=stop_words)),
                     ("clf", SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42))
                     ])

Wenn du keine eigene Methode angibst, werden die Daten automatisch stratifiziert mit `StratifiedKFold`. Das ist genau das, was du brauchst, deswegen brauchst du dich nicht weiter damit zu beschäftigen.

Genau überlegen musst du dir allerdings, welche Parameter du modizifieren bzw. durchsuchen möchtest. Dafür gibt es eine Konvention, und zwar gibst du als Präfix immer den Kurznamen des Pipeline-Schritts oben an, hinter zwei `__` kommt dann der zu variierende Parameter selbst als Schlüssel mit einem Tupel und den auszuprobierenden Werten als Wert:

In [ ]:
parameters = {
    "vect__min_df": (2, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),  
    "vect__use_idf": (True, False), 
    "vect__sublinear_tf": (True, False),
    "clf__alpha": (0.0001, 0.0002)
}

In diesem Fall würdest du also `3*2*2*2*2 = 48` Kombinationen ausprobieren.

## GridSearch mit Wiederholungen

Um die Stabilität zu überprüfen, führt `GridSearchCV` jeden Durchlauf dreimal mit unterschiedlichen Zusammensetzungen aus Trainings- und Testdaten durch. Damit ergeben sich 144 Durchläufe:

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(text_pipe, parameters, n_jobs=-1, cv=3, verbose=1, return_train_score=True)
grid_search.fit(adf["nav"], adf["author"].values)

Lass dir die besten Resultate ausgeben:

In [ ]:
print("Bester Score (hier Accuracy): %0.3f" % grid_search.best_score_)

print("Bestes Parameter Set:")
best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Das Ergebnis ist etwas überraschen, weil nämlich IDF gar nicht verwendet wird! Auch Bigramme scheinen keine Verbesserung zu bringen.

Du kannst dir die Resultate auch noch genauer in einem `DataFrame` darstellen lassen:

In [ ]:
pd.DataFrame(grid_search.cv_results_).sort_values("mean_test_score", ascending=False)

Wie du siehst, liegen die Ergebnisse sehr dicht beeinander. Resultat 28 mit IDF liegt z.B. nur wenig vom Optimum entfernt.

## `full_text` als alternatives Feld?

Du kannst jetzt alternativ mit dem gleichen Verfahren auc noch ein anderes Feld ausprobieren. z.B. den `full_text`: 

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(text_pipe, parameters, n_jobs=-1, cv=3, verbose=1, return_train_score=True)
grid_search.fit(adf["full_text"], adf["author"].values)

Lass dir die Ergebnisse anzeigen:

In [ ]:
print("Bester Score (hier Accuracy): %0.3f" % grid_search.best_score_)

print("Bestes Parameter Set:")
best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Das hat wesentlich besser funktioniert als die bisherige Klassifikation. Offenbar ist die Lemmatisierung und Konzentration auf bestimmte Wortarten der Autoren-Klassifikation abträglich.

## Verifikation und Detail-Analyse

Jetzt kannst du dir die Ergebnisse nochmal genau anschauen und für die oben gefundenen Parameter ein Training und eine Klassifikation durchführen:

In [ ]:
from sklearn.model_selection import train_test_split

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=10, sublinear_tf=True, use_idf=False)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["full_text"])

(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, adf["author"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=adf["author"].values)

clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, alpha=0.0002, random_state=42)
clf.fit(X_train, y_train)

Führe nun die Vorhersage durch und betrachte die Ergebnisse:

In [ ]:
from sklearn.metrics import classification_report

pred_test = clf.predict(X_test)
print(classification_report(y_test, pred_test))

Das sieht schon sehr gut aus, mit einem solchen Ergebnis kannst du fast immer zufrieden sein.

## Grid-Search automatisiert Optimierung

Mit einer Grid-Search kannst du versuchen, die Performance eines Klassifikators zu verbessern, in dem du stur alle möglichen Kombinationen der sog. *Hyperparameter* ausprobieren lässt. Das ist zwar nicht besonders effizient, funktioniert aber ziemlichh gut.

Wie du gesehen hast, waren die Ergebnisse bei der Autoren-Klassifikation sehr überraschend. Du hättest vorher bestimmt nicht gedacht, dass der Volltext ein besseres Ergebnis produziert als die linguistisch analysierten Texte. Erkläbar ist es trotzdem, nämlich mit dem Schreibstil der Autoren, der natürlich im Volltext am deutlichsten zutage tritt.